In [1]:
def dummy_func1(x, workflow_id=None):
    return x * 2


def dummy_func2(y, workflow_id=None):
    return y + y


def dummy_func3(z, w, workflow_id=None):
    return {"r": z + w}

In [2]:
def setup_local_dask_cluster():
    from dask.distributed import Client, LocalCluster
    from flowcept.flowceptor.plugins.dask.dask_plugins import (
        FlowceptDaskSchedulerPlugin,
        FlowceptDaskWorkerPlugin,
    )

    cluster = LocalCluster(n_workers=2)
    scheduler = cluster.scheduler
    client = Client(scheduler.address)

    # Instantiate and Register FlowceptPlugins
    scheduler_plugin = FlowceptDaskSchedulerPlugin(scheduler)
    scheduler.add_plugin(scheduler_plugin)

    worker_plugin = FlowceptDaskWorkerPlugin()
    client.register_worker_plugin(worker_plugin)

    return client

## Initializing Dask Cluster

In [4]:
dask_client = setup_local_dask_cluster()
dask_client

<Client: 'tcp://127.0.0.1:57682' processes=2 threads=10, memory=16.00 GiB>

## Initializing Flowcept Consumer

In [7]:
from flowcept.flowcept_api import FlowceptConsumerAPI
consumer = FlowceptConsumerAPI()
consumer.start()

Flowcept Consumer starting...
Ok, we're consuming messages!


## Running a simple workflow

In [9]:
import numpy as np
from uuid import uuid4
i1 = np.random.random()
i1 = np.random.random()
wf_id = f"wf_{uuid4()}"
print(f"Workflow_Id={wf_id}")
o1 = dask_client.submit(dummy_func1, i1, workflow_id=wf_id)
o2 = dask_client.submit(dummy_func2, o1, workflow_id=wf_id)
o3 = dask_client.submit(dummy_func3, o1, o2, workflow_id=wf_id)
print(f"Result={o3.result()}")
print(f"Task3_id={o3.key}")

Workflow_Id=wf_3295144b-01fe-44cb-b9b8-49cca066bd5b
Going to send to Redis an intercepted message:
	{"task_id": "dummy_func1-4e7423a713534ebf527de5550e3a51d8", "custom_metadata": {"scheduler": "tcp://127.0.0.1:57682", "scheduler_id": "Scheduler-10379ede-3c2a-493e-82a8-3a7df267b5b6", "scheduler_pid": 62516}, "status": "SUBMITTED", "utc_timestamp": 1675753962.155177, "activity_id": "dummy_func1", "used": {"arg0": 0.14897278819942383}, "workflow_id": "wf_3295144b-01fe-44cb-b9b8-49cca066bd5b", "plugin_id": "dask", "user": "root", "experiment_id": "super-experiment", "sys_name": "Darwin", "node_name": "MAC132633", "login_name": "rsr", "public_ip": "2600:6c5d:5b00:143e:749b:5ec9:25ca:79b6", "private_ip": "192.168.1.130"}
Going to send to Redis an intercepted message:
	{"task_id": "dummy_func2-5d57bb5889c08c6abd4fa765b149c39a", "custom_metadata": {"scheduler": "tcp://127.0.0.1:57682", "scheduler_id": "Scheduler-10379ede-3c2a-493e-82a8-3a7df267b5b6", "scheduler_pid": 62516}, "status": "SUBMITT

In [10]:
from flowcept.flowcept_api import TaskQueryAPI
query_api = TaskQueryAPI()

In [11]:
## Retrieving all tasks from this workflow

In [12]:
_filter = {"workflow_id": wf_id}
query_api.query(_filter)

[{'task_id': 'dummy_func1-4e7423a713534ebf527de5550e3a51d8',
  'custom_metadata': {'scheduler': 'tcp://127.0.0.1:57682',
   'scheduler_id': 'Scheduler-10379ede-3c2a-493e-82a8-3a7df267b5b6',
   'scheduler_pid': 62516},
  'status': 'FINISHED',
  'utc_timestamp': 1675753962.169221,
  'activity_id': 'dummy_func1',
  'used': {'arg0': 0.14897278819942383},
  'workflow_id': 'wf_3295144b-01fe-44cb-b9b8-49cca066bd5b',
  'plugin_id': 'dask',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2600:6c5d:5b00:143e:749b:5ec9:25ca:79b6',
  'private_ip': '192.168.1.130',
  'debug': True,
  'address': 'tcp://127.0.0.1:57689',
  'generated': {'arg0': 0.29794557639884767},
  'start_time': 1675735962.169018,
  'end_time': 1675735962.169022},
 {'task_id': 'dummy_func2-5d57bb5889c08c6abd4fa765b149c39a',
  'custom_metadata': {'scheduler': 'tcp://127.0.0.1:57682',
   'scheduler_id': 'Scheduler-10379ede-3c2a-493e-8

## Retrieving only task3

In [14]:
_filter = {"task_id": o3.key}
query_api.query(_filter)

[{'task_id': 'dummy_func3-7b071fc6108e812f4f43ce2e400e5cf6',
  'custom_metadata': {'scheduler': 'tcp://127.0.0.1:57682',
   'scheduler_id': 'Scheduler-10379ede-3c2a-493e-82a8-3a7df267b5b6',
   'scheduler_pid': 62516},
  'status': 'FINISHED',
  'utc_timestamp': 1675753962.178617,
  'dependencies': ['dummy_func1-4e7423a713534ebf527de5550e3a51d8',
   'dummy_func2-5d57bb5889c08c6abd4fa765b149c39a'],
  'activity_id': 'dummy_func3',
  'used': {'arg0': 'dummy_func1-4e7423a713534ebf527de5550e3a51d8',
   'arg1': 'dummy_func2-5d57bb5889c08c6abd4fa765b149c39a'},
  'workflow_id': 'wf_3295144b-01fe-44cb-b9b8-49cca066bd5b',
  'plugin_id': 'dask',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2600:6c5d:5b00:143e:749b:5ec9:25ca:79b6',
  'private_ip': '192.168.1.130',
  'debug': True,
  'address': 'tcp://127.0.0.1:57689',
  'generated': {'r': 0.893836729196543},
  'start_time': 1675735962.1783652,
  'e